In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.preprocessing import MinMaxScaler
import time

In [6]:
from google.colab import files

uploaded = files.upload()

Saving 03.npz to 03.npz
Saving label.npy to label.npy
Saving 02.npz to 02.npz
Saving 01.npz to 01.npz


In [0]:
def window_slice(data, time_steps):
    data = np.transpose(data, (1, 0, 2)).reshape(-1, 310)
    xs = []
    for i in range(data.shape[0] - time_steps + 1):
        xs.append(data[i: i + time_steps])
    xs = np.concatenate(xs).reshape((len(xs), -1, 310))
    return xs

    
def load_data(data_file, label_file, time_steps):
    datas = np.load(data_file)
    labels = np.load(label_file)
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    for data, label in zip([datas[k] for k in datas.keys()[:9]], list(labels[:9])):
        X_train.append(window_slice(data, time_steps))
        y_train.extend([label] * len(X_train[-1]))
    for data, label in zip([datas[k] for k in datas.keys()[9:]], list(labels[9:])):
        X_test.append(window_slice(data, time_steps))
        y_test.extend([label] * len(X_test[-1]))
    return np.concatenate(X_train), np.concatenate(X_test), np.array(y_train), np.array(y_test)

In [0]:
#unrolled through 28 time steps
time_steps=50
#hidden LSTM units
num_units=128
#rows of 28 pixels
features=310
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=3

In [0]:
X_train1, X_test1, y_train1, y_test1 = load_data('01.npz', 'label.npy', time_steps)
X_train2, X_test2, y_train2, y_test2 = load_data('02.npz', 'label.npy', time_steps)
X_train3, X_test3, y_train3, y_test3 = load_data('03.npz', 'label.npy', time_steps)

In [0]:
X_train = np.concatenate((X_train1, X_train2, X_train3))
X_test = np.concatenate((X_test1, X_test2, X_test3))
y_train = np.concatenate((y_train1, y_train2, y_train3))
y_test = np.concatenate((y_test1, y_test2, y_test3))
# X_train, X_test, y_train, y_test = X_train2, X_test2, y_train2, y_test2

In [0]:
scaler = MinMaxScaler(copy=False)
X_train = X_train.reshape((-1, 310))
X_test = X_test.reshape((-1, 310))
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train = X_train.reshape((-1, time_steps, 310))
X_test = X_test.reshape((-1, time_steps, 310))

In [0]:
#size of batch
batch_size=100
epoch = 30
max_steps = epoch * X_train.shape[0] // batch_size

In [0]:
#weights and biases of appropriate shape to accomplish above task
weights=tf.Variable(tf.random_normal([num_units,n_classes]))
bias=tf.Variable(tf.random_normal([n_classes]))
#defining placeholders
#input image placeholder
data_placeholder=tf.placeholder(tf.float32,[None,time_steps,features])
#input label placeholder
label_placeholder=tf.placeholder(tf.int64,[None])

In [0]:
input=tf.unstack(data_placeholder ,time_steps,1)
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype=tf.float32)
keep_prob = tf.placeholder(tf.float32)
final_output = tf.nn.dropout(outputs[-1], keep_prob)

In [0]:
logits=tf.matmul(final_output,weights)+bias
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
  labels=label_placeholder))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logits, 1), label_placeholder)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
beginTime = time.time()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # Initialize variables
    sess.run(tf.global_variables_initializer())

    # Repeat max_steps times
    for i in range(max_steps):

        # Generate input data batch
        indices = np.random.choice(X_train.shape[0], batch_size)
        data_batch = X_train[indices]
        label_batch = y_train[indices]

        # Periodically print out the model's current accuracy
        if i * batch_size // X_train.shape[0] != (i - 1) * batch_size // X_train.shape[0]:
#         if i * batch_size // X_train.shape[0] != (i - 1) * batch_size // X_train.shape[0] and (i * batch_size // X_train.shape[0]) % 5 == 0:
#             train_accuracy = sess.run(accuracy, feed_dict={
#                 data_placeholder: data_batch, label_placeholder: label_batch})
#             print('Step {:5d}: training accuracy {:g}'.format(i, train_accuracy))
            test_accuracy = sess.run(accuracy, feed_dict={
                data_placeholder: X_test,
                label_placeholder: y_test,
                keep_prob: 1.0})
            print('Epoch {:3d}, Test accuracy {:g}'.format(i * batch_size // X_train.shape[0], test_accuracy))

        # Perform a single training step
        sess.run(train_step, feed_dict={data_placeholder: data_batch,
                                        label_placeholder: label_batch,
                                        keep_prob: 0.5})

    # After finishing the training, evaluate on the test set
    test_accuracy = sess.run(accuracy, feed_dict={
        data_placeholder: X_test,
        label_placeholder: y_test,
        keep_prob: 1.0})
    print('Test accuracy {:g}'.format(test_accuracy))

endTime = time.time()
print('Total time: {:5.2f}s'.format(endTime - beginTime))

Epoch   0, Test accuracy 0.270642
Epoch   1, Test accuracy 0.745872
Epoch   2, Test accuracy 0.908868
Epoch   3, Test accuracy 0.896024
Epoch   4, Test accuracy 0.942202
Epoch   5, Test accuracy 0.942202
Epoch   6, Test accuracy 0.942202
Epoch   7, Test accuracy 0.942202
Epoch   8, Test accuracy 0.944037
Epoch   9, Test accuracy 0.942202
Epoch  10, Test accuracy 0.942202
Epoch  11, Test accuracy 0.942202
Epoch  12, Test accuracy 0.942202
Epoch  13, Test accuracy 0.942202
Epoch  14, Test accuracy 0.942202
Epoch  15, Test accuracy 0.942202
Epoch  16, Test accuracy 0.942202
Epoch  17, Test accuracy 0.942813
Epoch  18, Test accuracy 0.942202
Epoch  19, Test accuracy 0.942202
Epoch  20, Test accuracy 0.942202
Epoch  21, Test accuracy 0.942202
Epoch  22, Test accuracy 0.942202
Epoch  23, Test accuracy 0.942202
Epoch  24, Test accuracy 0.942202
Epoch  25, Test accuracy 0.942202
Epoch  26, Test accuracy 0.942202
Epoch  27, Test accuracy 0.942202
Epoch  28, Test accuracy 0.942202
Epoch  29, Tes